In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [9]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
ratings = pd.read_csv("ratings.csv")
keywords = pd.read_csv("keywords.csv")
links = pd.read_csv("links.csv")
metadata = pd.read_csv("movies_metadata.csv")


/var/folders/vq/r13xv8cj2gj29pdtj9rfqt4h0000gn/T/ipykernel_95805/1663131610.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("movies_metadata.csv")


In [7]:
credits.shape

(4803, 4)

In [8]:
movies.shape

(4803, 20)

In [10]:
ratings

,version https://git-lfs.github.com/spec/v1
0,oid sha256:5c6e8af7e8f71247208cf3fce4a7d52e61e...
1,size 709550327
